In [1]:
import dotenv
import os
import json
from typing import Dict, List
import pandas as pd
import datetime
from sqlalchemy import create_engine, text

dotenv.load_dotenv()

UTC_OFFSET = int(os.getenv("UTC_OFFSET"))

# Helper functions

In [2]:
def get_new_data(file_name: str, table_info: Dict) -> pd.DataFrame:
    # Get the data from the csv file

    path = (
        os.path.join(os.getenv("IMPORT_CSV_PATH"), file_name)
        .encode("latin1")
        .decode("utf-8")
    )

    csv_info = get_csv_info(table_info)

    assert os.path.exists(path), f"File '{path}' does not exist."
    return pd.read_csv(
        path,
        sep=csv_info["delimiter"],
        encoding=csv_info["encoding"],
    )


def get_db_info(table_name: str, db_info: Dict) -> Dict:
    for table in db_info:
        if table["table_name"] == table_name:
            return table

    raise ValueError(f"Table {table_name} not found in db_info")


def load_tables() -> Dict[str, str]:
    return json.load(
        open(
            os.path.join(
                os.getenv("EPIC_2_SAVE_LOCATION"), os.getenv("TABLES_FILE_NAME")
            )
        )
    )


def load_info():
    return json.load(
        open(
            os.path.join(
                os.getenv("EPIC_2_SAVE_LOCATION"), os.getenv("DB_INFO_FILE_NAME")
            )
        )
    )


def get_csv_info(table_info) -> Dict:
    # Set default return value
    return_value = {
        "encoding": "utf-8",
        "delimiter": ",",
    }
    # Check if the table has a custom encoding
    if "info" not in table_info:
        return return_value
    # Check if the encoding is set
    for key in return_value.keys():
        if key in table_info["info"]:
            return_value[key] = table_info["info"][key]
    return return_value


def to_date(string) -> datetime.date or None:
    # Make sure the string is formatted correctly
    # Different way of dealing with unformatted dates -- Priority: Low
    string = str(string).strip()
    if string is None or len(string.split("-")) != 3:
        return None
    return datetime.datetime.strptime(string, "%d-%m-%Y").date()


def to_float(string) -> float or None:
    string = str(string).strip()
    string = string.replace(",", ".")
    try:
        return float(string)
    except ValueError:
        return None


def to_datetime(string) -> datetime.datetime or None:
    string = str(string).strip()

    if string is None or len(string.split("-")) != 3:
        return None
    return datetime.datetime.strptime(string, "%d-%m-%Y %H:%M:%S")


def to_datetime_utc(string) -> datetime.datetime or None:
    string = str(string).lower()
    # Remove optional (UTC) from the string
    assert "(utc)" in string, f"String '{string}' might not be utc."
    string = string.replace("(utc)", "")
    string = str(string).strip()

    if string is None or len(string.split("-")) != 3:
        return None

    utc_datetime = datetime.datetime.strptime(string, "%m-%d-%Y %H:%M:%S")
    # Local datetime is Brussels time
    local_datetime = utc_datetime + datetime.timedelta(hours=UTC_OFFSET)
    return local_datetime


def ja_nee(string: str) -> int:
    assert string.lower() in ["ja", "nee"], f"String '{string}' is not 'ja' or 'nee'."
    return 1 if string.lower() == "ja" else (0 if string.lower() == "nee" else None)


def clean_nan(string) -> str or None:
    string = str(string).strip()
    if string is None:
        return None
    if string.lower() == "nan":
        return None
    return string.strip()


def create_conn():
    driver = os.getenv("DB_DRIVER")
    server = os.getenv("DB_SERVER")
    database = os.getenv("DB_NAME")
    trusted_connection = os.getenv("DB_TRUSTED_CONNECTION")

    return create_engine(
        f"mssql+pyodbc://{server}/{database}?trusted_connection={trusted_connection}&driver={driver}"
    )

In [3]:
conn = create_conn()

info = load_info()
for value in info:
    value["table_name"] = str(value["table_name"]).encode("latin1").decode("utf-8")
info

[{'table_name': 'account',
  'columns': {'crm_Account_Account': 'crm_account_id',
   'crm_account_adres_geografische_regio': 'crm_account_adres_geografische_regio',
   'crm_account_adres_geografische_subregio': 'crm_account_adres_geografische_subregio',
   'crm_account_adres_plaats': 'crm_account_adres_plaats',
   'crm_account_adres_postcode': 'crm_account_adres_postcode',
   'crm_account_adres_provincie': 'crm_account_adres_provincie',
   'crm_Account_Adres_Land': 'crm_account_adres_land',
   'crm_Account_Industriezone_Naam_': 'crm_account_industriezone_naam',
   'crm_account_is_voka_entiteit': 'crm_account_is_voka_entiteit',
   'crm_account_ondernemingsaard': 'crm_account_ondernemingsaard',
   'crm_account_ondernemingstype': 'crm_account_ondernemingstype',
   'crm_account_oprichtingsdatum': 'crm_account_oprichtingsdatum',
   'crm_account_primaire_activiteit': 'crm_account_primaire_activiteit',
   'crm_account_reden_van_status': 'crm_account_reden_van_status',
   'crm_account_status':

In [4]:
tables = load_tables()
tables

{'Account activiteitscode.csv': 'account_activiteitscode',
 'Account.csv': 'account',
 'Activiteit vereist contact.csv': 'activiteit_vereist_contact',
 'Activiteitscode.csv': 'activiteitscode',
 'Afspraak alle.csv': 'afspraak_alle',
 'Afspraak betreft account_cleaned.csv': 'afspraak_betreft_account',
 'Afspraak betreft contact_cleaned.csv': 'afspraak_betreft_contact',
 'Afspraak_account_gelinkt_cleaned.csv': 'afspraak_account_gelinkt',
 'CDI mailing.csv': 'cdi_mailing',
 'CDI sent email clicks.csv': 'cdi_sentemail_kliks',
 'CDI visits.csv': 'cdi_visit',
 'Campagne.csv': 'campagne',
 'Contac functie.csv': 'contactfunctie',
 'Contact.csv': 'contact',
 'FinanciÃ«le data.csv': 'financiele_data',
 'Functie.csv': 'functie',
 'Gebruikers.csv': 'gebruikers',
 'Info en klachten.csv': 'info_en_klachten',
 'Inschrijving.csv': 'inschrijving',
 'Lidmaatschap.csv': 'lidmaatschap',
 'Persoon.csv': 'persoon',
 'Sessie inschrijving.csv': 'sessie_inschrijving',
 'Sessie.csv': 'sessie',
 'teams.csv': 'te

In [5]:
table_info = get_db_info("cdi_sentemail_kliks", info)
sent_email = get_new_data("CDI sent email clicks.csv", table_info)
sent_email

,crm_CDI_SentEmail_kliks_Clicks,crm_CDI_SentEmail_kliks_Contact,crm_CDI_SentEmail_kliks_E_mail_versturen,crm_CDI_SentEmail_kliks_Sent_Email
0,1,0004E1C6-23E3-EC11-BB3C-000D3ABD8BDA,F206630D-096D-EE11-9AE7-6045BD895D85,37AEFCD3-C0E2-4BF0-92C7-65E4BB7DA48E
1,1,000BA88A-1B96-ED11-AAD1-6045BD895D85,F4B80760-4864-EE11-8DF0-6045BD895805,2E138D92-1907-4F04-811D-F757F9E71C4E
2,1,00113F8D-6A5A-E811-80F1-001DD8B72B61,23AE5234-FE73-EE11-8179-6045BD895D0D,BA25A1BA-BEE5-40F9-ACC8-0F3BA9B62E9B
3,1,0011C821-5BD4-ED11-A7C7-6045BD895BFB,31CEA13E-0156-EE11-BE6E-6045BD895420,44BDEE5A-7EC8-4671-B556-E0EAE9578A46
4,1,0011C821-5BD4-ED11-A7C7-6045BD895BFB,4D3A8A9B-A25B-EE11-BE6E-6045BD895BFB,7C955637-9236-4822-B231-911A9A9BE8E8
...,...,...,...,...
41880,104,B71D45D1-10DD-ED11-A7C6-000D3A4AB78E,FF46C892-0F5D-EE11-BE6F-6045BD895499,87D8C65B-BFD9-4045-8512-ACC3B8AFD4A7
41881,107,1DE537A7-0206-E511-ABE8-005056B06EB4,4A2DDAE0-4167-EE11-9AE7-6045BD895D78,54612D3D-10CF-4932-8163-11005BDA8A20
41882,242,38549C3C-AAC3-EB11-8120-001DD8B72B61,640D4D20-7266-EE11-9AE7-6045BD895FE3,EBDFE950-D465-41BC-B7A1-FB76A27894BA
41883,356,38549C3C-AAC3-EB11-8120-001DD8B72B61,53A6E4A2-B071-EE11-8179-6045BD895A72,AF270A83-09D1-4A51-908C-355D12899FD1


In [6]:

contact_df = pd.read_sql("SELECT * FROM contact", conn)
contact_df

,crm_contact_id,crm_contact_account,crm_contact_functietitel,crm_persoon_id,crm_contact_status,crm_contact_voka_medewerker
0,E14A8BC8-DCC9-EC11-A7B5-000D3A20A90F,7D15D389-FF2A-E511-93F3-005056B06EC4,CLB arts coordinator,EE4A8BC8-DCC9-EC11-A7B5-000D3A20A90F,Actief,0
1,B548B872-DDC9-EC11-A7B5-000D3A20A90F,E82DAACF-A656-EC11-8F8F-000D3AAAAFCA,Co-Founder,8E77DD5A-7276-EB11-811D-001DD8B72B62,Actief,0
2,9DBB5B1D-DFC9-EC11-A7B5-000D3A20A90F,9EBB5B1D-DFC9-EC11-A7B5-000D3A20A90F,founder of Tablot,F1DF8A23-DFC9-EC11-A7B5-000D3A20A90F,Actief,0
3,1BE002B3-E3C9-EC11-A7B5-000D3A20A90F,A9EA2C80-AC68-E111-B43A-00505680000A,"HR Coördinator WOLL (Welzijn, Ontwikkeling, Le...",1FE002B3-E3C9-EC11-A7B5-000D3A20A90F,Inactief,0
4,805A8764-EAC9-EC11-A7B5-000D3A20A90F,FBC1E08F-B368-E111-B43A-00505680000A,HR Manager,6CE11D6E-4969-E111-B43A-00505680000A,Actief,0
...,...,...,...,...,...,...
1173102,C19AE6AE-585A-4D5D-B8E7-FF92D882BA15,77990022-BA68-E111-B43A-00505680000A,Partner,C12FD79C-EB67-E111-A00F-00505680000A,Actief,0
1173103,AB20792C-9ACE-44DA-B4DB-FFA987F1D608,64036EE0-9E68-E111-B43A-00505680000A,Planningverantwoordelijke,88F84387-4398-4FD7-B579-014471B84267,Inactief,0
1173104,DF4228E0-82BD-4374-BED2-FFB79BC429B4,52D13234-0669-E111-B43A-00505680000A,Team Leader Gemeentelijke Infra,04E69AD1-9FA8-49AE-AF0D-7966D1A8DE72,Actief,0
1173105,8E3FF0E2-14CD-483D-AC00-FFE2CFB5E136,4E996FAB-0869-E111-B43A-00505680000A,woordvoerder bpost,E5FA1050-349C-4D35-8C3E-5BD42C72D0BD,Inactief,0


In [7]:
# Overlap
df_merged = pd.merge(
    sent_email,
    contact_df,
    how="inner",
    left_on="crm_CDI_SentEmail_kliks_Contact",
    right_on="crm_contact_id",
)
df_merged

,crm_CDI_SentEmail_kliks_Clicks,crm_CDI_SentEmail_kliks_Contact,crm_CDI_SentEmail_kliks_E_mail_versturen,crm_CDI_SentEmail_kliks_Sent_Email,crm_contact_id,crm_contact_account,crm_contact_functietitel,crm_persoon_id,crm_contact_status,crm_contact_voka_medewerker
0,1,0004E1C6-23E3-EC11-BB3C-000D3ABD8BDA,F206630D-096D-EE11-9AE7-6045BD895D85,37AEFCD3-C0E2-4BF0-92C7-65E4BB7DA48E,0004E1C6-23E3-EC11-BB3C-000D3ABD8BDA,302612A3-B568-E111-B43A-00505680000A,Private banker,FE03E1C6-23E3-EC11-BB3C-000D3ABD8BDA,Actief,0
1,4,0004E1C6-23E3-EC11-BB3C-000D3ABD8BDA,B3C53531-A769-EE11-9AE7-6045BD895420,EDD22253-A831-425D-B10D-131563385885,0004E1C6-23E3-EC11-BB3C-000D3ABD8BDA,302612A3-B568-E111-B43A-00505680000A,Private banker,FE03E1C6-23E3-EC11-BB3C-000D3ABD8BDA,Actief,0
2,1,000BA88A-1B96-ED11-AAD1-6045BD895D85,F4B80760-4864-EE11-8DF0-6045BD895805,2E138D92-1907-4F04-811D-F757F9E71C4E,000BA88A-1B96-ED11-AAD1-6045BD895D85,32308AEC-6CDB-E811-80F9-001DD8B72B62,Senior Corporate banker,18E6C190-1B96-ED11-AAD1-6045BD895D85,Actief,0
3,1,00113F8D-6A5A-E811-80F1-001DD8B72B61,23AE5234-FE73-EE11-8179-6045BD895D0D,BA25A1BA-BEE5-40F9-ACC8-0F3BA9B62E9B,00113F8D-6A5A-E811-80F1-001DD8B72B61,3DA501DD-B367-E111-A00F-00505680000A,Productieleider,8F737173-6A5A-E811-80F1-001DD8B72B61,Actief,0
4,1,0011C821-5BD4-ED11-A7C7-6045BD895BFB,31CEA13E-0156-EE11-BE6E-6045BD895420,44BDEE5A-7EC8-4671-B556-E0EAE9578A46,0011C821-5BD4-ED11-A7C7-6045BD895BFB,A1D6F809-45B7-ED11-83FF-6045BD8952CE,Zaakvoerder,0711C821-5BD4-ED11-A7C7-6045BD895BFB,Actief,0
...,...,...,...,...,...,...,...,...,...,...
40600,64,E484CF76-1FBC-ED11-83FF-6045BD895BFB,B2237C92-DC5D-EE11-BE6F-6045BD895FE3,45BAA8FE-EB97-48BD-9DA9-E62D86CD3FE2,E484CF76-1FBC-ED11-83FF-6045BD895BFB,5697C274-C568-E111-B43A-00505680000A,Coördinator Young Voka Metropolitan | Administ...,0853EB4B-1EBC-ED11-83FF-6045BD895BFB,Actief,0
40601,107,1DE537A7-0206-E511-ABE8-005056B06EB4,4A2DDAE0-4167-EE11-9AE7-6045BD895D78,54612D3D-10CF-4932-8163-11005BDA8A20,1DE537A7-0206-E511-ABE8-005056B06EB4,F6A86F9D-C668-E111-B43A-00505680000A,Adviseur milieu en duurzaamheid Alfaport Voka,0C52C98E-0C68-E111-A00F-00505680000A,Actief,1
40602,242,38549C3C-AAC3-EB11-8120-001DD8B72B61,640D4D20-7266-EE11-9AE7-6045BD895FE3,EBDFE950-D465-41BC-B7A1-FB76A27894BA,38549C3C-AAC3-EB11-8120-001DD8B72B61,99C841DF-C768-E111-B43A-00505680000A,Projectmanager duurzaam ondernemen,9C26555B-FF7A-E911-80FE-001DD8B72B61,Actief,1
40603,356,38549C3C-AAC3-EB11-8120-001DD8B72B61,53A6E4A2-B071-EE11-8179-6045BD895A72,AF270A83-09D1-4A51-908C-355D12899FD1,38549C3C-AAC3-EB11-8120-001DD8B72B61,99C841DF-C768-E111-B43A-00505680000A,Projectmanager duurzaam ondernemen,9C26555B-FF7A-E911-80FE-001DD8B72B61,Actief,1


In [8]:
table_info = get_db_info("cdi_visit", info)
sent_email = get_new_data("CDI visits.csv", table_info)
sent_email

,crm_CDI_Visit_Adobe_Reader,crm_CDI_Visit_Bounce,crm_CDI_Visit_Browser,crm_CDI_Visit_Campagne_Code,crm_CDI_Visit_Campaign,crm_CDI_Visit_IP_Stad,crm_CDI_Visit_IP_Company,crm_CDI_Visit_Contact,crm_CDI_Visit_Contact_Naam_,crm_CDI_Visit_containssocialprofile,...,crm_CDI_Visit_Referring_Host,crm_CDI_Visit_Score,crm_CDI_Visit_Referrer_Type,crm_CDI_Visit_Started_On,crm_CDI_Visit_IP_Status,crm_CDI_Visit_Time,crm_CDI_Visit_Total_Pages,crm_CDI_Visit_Visit,crm_CDI_Visit_Aangemaakt_op,crm_CDI_Visit_Gewijzigd_op
0,Ja,Nee,SAFARI,NaN,2E3181C5-92E8-EB11-8121-001DD8B72B61,Antwerp,Telenet,5FC1E5CB-A86B-E111-B43A-00505680000A,"Poels, Dirk",Nee,...,NaN,5,E-mail,7-11-2021 06:16:41,VLG,11-07-2021 06:16:41 (UTC),2,92B18E10-6870-4046-A0DE-E3A09A85C049,7-11-2021 07:16:42,28-10-2023 17:21:35
1,Nee,Ja,NaN,NaN,NaN,Weerde,Telenet,10D29432-805E-EC11-8F8F-000D3A2BC671,"Petré, Ulrich",Nee,...,www.linkedin.com,4,E-mail,26-3-2022 06:50:50,VLG,03-26-2022 06:50:50 (UTC),1,4AD39F7F-A623-4730-A51C-6DEF3EFE9C06,26-3-2022 07:51:12,14-10-2023 16:39:55
2,Nee,Ja,NaN,NaN,D812EBE5-0768-EE11-9AE7-6045BD895D85,Jabbeke,Telenet,0E97FD13-51C1-E311-9A5C-005056B06EB4,"Van Wesemael, Jan",Nee,...,www.linkedin.com,4,E-mail,21-10-2023 11:11:20,VLG,10-21-2023 11:11:20 (UTC),1,E7C45C89-9E8E-4AF6-8229-1A2D71FA3CB1,24-10-2023 02:57:12,24-10-2023 02:57:14
3,Nee,Ja,ANDROID,NaN,NaN,NaN,NaN,086113C8-3D73-E111-B43A-00505680000A,"Tuybens, Dieter",Nee,...,NaN,4,E-mail,8-9-2023 07:50:48,NaN,09-08-2023 07:50:48 (UTC),1,808370D9-46E0-48BD-A074-DBA8123B99CC,8-9-2023 09:51:06,14-10-2023 16:47:36
4,Nee,Ja,ANDROID,NaN,NaN,NaN,NaN,502642E5-5CC9-E411-81DA-005056B06EB4,"Martens, Els",Nee,...,NaN,4,E-mail,15-9-2023 06:04:10,NaN,09-15-2023 06:04:10 (UTC),1,E765C779-0DA4-4C6D-A05E-F6BC77EFEA4B,15-9-2023 08:04:12,7-10-2023 16:33:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62260,Nee,Nee,SAFARI,NaN,FD0E0E2C-DF7A-ED11-81AD-6045BD895D85,Halle,Telenet,DA2B3EB4-A41B-E811-80F0-001DD8B72B61,"Verhoeven, Michele",Nee,...,NaN,14,E-mail,6-2-2023 13:06:30,VLG,02-06-2023 13:06:30 (UTC),11,2FA5FA7A-A2B9-4D68-88D0-88099A32C047,13-2-2023 02:45:07,21-10-2023 16:33:33
62261,Nee,Nee,SAFARI,NaN,FDDD8E61-4AA2-ED11-AAD1-6045BD895B5A,Wilsele,Belgacom SA,B9FF45B9-AB4C-E511-B697-005056B06EB4,"Cox, Pieter",Nee,...,www.voka.be,10,Onbekend,27-2-2023 10:33:42,VLG,02-27-2023 10:34:10 (UTC),7,C1DB5C01-8E84-4CAB-B4E8-98E57FC5790A,27-2-2023 17:05:26,7-10-2023 16:31:34
62262,Nee,Nee,SAFARI,NaN,FE2BCE78-CCC3-ED11-83FF-6045BD8950C6,Leuven,PRO,5A9B8938-FC4D-EE11-BE6E-6045BD895FE4,"Mihoc, Claudia",Nee,...,NaN,7,Direct verkeer,18-10-2023 14:18:50,VLG,10-18-2023 14:19:10 (UTC),2,41387218-1436-43B0-9A2D-019D4648363B,18-10-2023 17:49:07,18-10-2023 17:49:44
62263,Nee,Nee,SAFARI,NaN,FE3D13D2-7B1B-EC11-8123-001DD8B72B61,Vilvoorde,Telenet,D01F4444-8A51-EB11-811A-001DD8B72B62,"Daniëls, Petra",Nee,...,NaN,6,E-mail,1-11-2021 09:18:50,VLG,11-01-2021 09:18:50 (UTC),3,46952633-0996-4700-B400-CD28846015A3,1-11-2021 10:19:26,7-10-2023 17:25:55


In [9]:
# Check pageviews

table_info = get_db_info("cdi_pageview", info)
df = get_new_data("CDI pageviews.csv", table_info)

df

C:\Users\Maarten Boon\AppData\Local\Temp\ipykernel_22616\2136661391.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(


,crm_CDI_PageView_Anonymous_Visitor,crm_CDI_PageView_Browser,crm_CDI_PageView_Campaign,crm_CDI_PageView_Contact,crm_CDI_PageView_Duration,crm_CDI_PageView_Operating_System,crm_CDI_PageView_Page_View,crm_CDI_PageView_Referrer_Type,crm_CDI_PageView_Time,crm_CDI_PageView_Page_Title,...,crm_CDI_PageView_Url,crm_CDI_PageView_Viewed_On,crm_CDI_PageView_Visit,crm_CDI_PageView_Visitor_Key,crm_CDI_PageView_Web_Content,crm_CDI_PageView_Aangemaakt_op,crm_CDI_PageView_Gewijzigd_door,crm_CDI_PageView_Gewijzigd_op,crm_CDI_PageView_Status,crm_CDI_PageView_Reden_van_status
0,NaN,NaN,NaN,10D29432-805E-EC11-8F8F-000D3A2BC671,NaN,MAC,44335D13-04FD-436D-8584-558EC737286D,Email,03-26-2022 06:50:50 (UTC),Ondernemers Vlaams-Brabant 2022 - Digitaliseri...,...,https://www.voka.be/publicaties/ondernemers-vl...,26-3-2022 06:50:50,4AD39F7F-A623-4730-A51C-6DEF3EFE9C06,c3f6c4816cc445e1925389f109ba0c72,NaN,26-3-2022 07:51:12,C135D6C5-075E-E111-B3B9-18A905732A15,14-10-2023 16:39:55,Actief,Actief
1,NaN,NaN,D43CF03F-4652-EE11-BE6D-6045BD895BFB,36FFA768-E494-EA11-8111-001DD8B72B62,NaN,MAC,82E6AF8D-9B27-4267-A57A-CD0186D5A8B9,Email,10-02-2023 12:30:03 (UTC),Aan de slag met CSRD | Voka,...,https://www.voka.be/activiteiten/aan-de-slag-m...,2-10-2023 12:30:03,D4D395FA-8A9C-4072-A935-08E920D5A3F9,cddbec4213e14a8d9ad173f0d8c21e90,NaN,2-10-2023 14:32:27,A30BACB8-C3A9-403D-9C1E-463FDD2C5AFC,2-10-2023 14:32:29,Actief,Actief
2,NaN,NaN,D812EBE5-0768-EE11-9AE7-6045BD895D85,0E97FD13-51C1-E311-9A5C-005056B06EB4,NaN,MAC,9E7F549E-420B-4AB0-AECF-B160126F607D,Email,10-21-2023 11:11:20 (UTC),Digitale douanetoepassingen - wat komt er op j...,...,https://www.voka.be/activiteiten/digitale-doua...,21-10-2023 11:11:20,E7C45C89-9E8E-4AF6-8229-1A2D71FA3CB1,c2d49b78f3dd4b9fa37b7a271eae4b03,NaN,24-10-2023 02:57:13,A30BACB8-C3A9-403D-9C1E-463FDD2C5AFC,24-10-2023 02:57:14,Actief,Actief
3,NaN,ANDROID,NaN,006E1700-3DB5-E911-8104-001DD8B72B61,NaN,LINUX,4079C95B-0F14-4092-90A6-37EABACACA08,Email,10-04-2023 12:13:31 (UTC),Oproep Voka: Doorzetten met aangepast stikstof...,...,https://www.voka.be/nieuws/oproep-voka-doorzet...,4-10-2023 12:13:31,9A3D9C41-C32E-422F-8807-B12A29F74516,c476076f065c446f9837d5c71c3f8470,NaN,4-10-2023 14:15:06,A30BACB8-C3A9-403D-9C1E-463FDD2C5AFC,4-10-2023 14:15:08,Actief,Actief
4,NaN,ANDROID,NaN,00FBC182-F40E-E311-828B-005056B06EB4,NaN,LINUX,AF0F3F69-A798-4754-9570-D93F1F0F6B4A,Email,06-14-2023 06:35:23 (UTC),Gentse start-up Those Vegan Cowboys ontwikkelt...,...,https://www.voka.be/nieuws/gentse-start-those-...,14-6-2023 06:35:23,6C54CFDC-DCA7-4109-A5F3-BFA6BB7B87AF,c6db77fca8134186b14bae25a35d5880,NaN,14-6-2023 08:35:24,C135D6C5-075E-E111-B3B9-18A905732A15,21-10-2023 16:31:41,Actief,Actief
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184186,NaN,SAFARI,FFB6A1EC-9A6F-EC11-8943-000D3A2790B4,DA2B3EB4-A41B-E811-80F0-001DD8B72B61,33.0,MAC,2020798E-A05F-46BC-B013-C00032E42EF0,Referring Sites,10-10-2022 12:44:19 (UTC),Schrijf in voor Update Routeplan 2025 (GRATIS)...,...,https://www.voka.be/node/35676/event-subscribe...,10-10-2022 12:44:19,DC2C2E54-F443-4639-91F4-0041A2EA244D,ca6c92adf49845be903c02f4d66779f8,NaN,11-10-2022 00:52:29,C135D6C5-075E-E111-B3B9-18A905732A15,21-10-2023 16:33:31,Actief,Actief
184187,NaN,SAFARI,FFB6A1EC-9A6F-EC11-8943-000D3A2790B4,DA2B3EB4-A41B-E811-80F0-001DD8B72B61,47.0,MAC,BE393C78-E81C-4CB8-9DD9-F02DC667FC1B,Referring Sites,10-10-2022 12:43:32 (UTC),Schrijf in voor Update Routeplan 2025 (GRATIS)...,...,https://www.voka.be/node/35676/event-subscribe...,10-10-2022 12:43:32,DC2C2E54-F443-4639-91F4-0041A2EA244D,ca6c92adf49845be903c02f4d66779f8,NaN,11-10-2022 00:52:26,C135D6C5-075E-E111-B3B9-18A905732A15,21-10-2023 16:33:32,Actief,Actief
184188,NaN,SAFARI,FFB6A1EC-9A6F-EC11-8943-000D3A2790B4,DA2B3EB4-A41B-E811-80F0-001DD8B72B61,335.0,MAC,847BBE6D-7428-4BF5-83BD-B46FBB5C98F3,Referring Sites,10-10-2022 12:24:21 (UTC),Update Routeplan 2025 (GRATIS) | Voka,...,https://www.voka.be/activiteiten/update-routep...,10-10-

In [10]:
# Load the necessary tables
campaign_df = pd.read_sql("SELECT * FROM campagne", conn)
campaign_df

,crm_campagne_id,crm_campagne_campagne_nr,crm_campagne_einddatum,crm_campagne_naam,crm_campagne_naam_in_email,crm_campagne_reden_van_status,crm_campagne_startdatum,crm_campagne_status,crm_campagne_type_campagne,crm_campagne_url_voka_be,crm_campagne_soort_campagne
0,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,22-OV-06-0017,2022-06-15 12:00:00,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,Voka's Quick Refresh - Energiedelen en energie...,Voltooid,2022-06-15 11:00:00,Actief,Infosessie,https://www.voka.be/quickrefresh,Online
1,1B445E22-A9CB-EC11-A7B5-000D3A20A90F,22-OV-02-0069,2022-06-24 14:00:00,OV-Infosessie - Bedrijfsoverdracht: Wat met de...,Infosessie - Bedrijfsoverdracht: Wat met de ov...,Nieuw,2022-06-24 09:00:00,Actief,Opleiding,None,Offline
2,12856972-10CD-EC11-A7B5-000D3A20BE4E,22-OV-01-0103,2022-06-23 23:00:00,OV-AO-HR Midzomerbar,HR Midzomerbar,Nieuw,2022-06-23 18:00:00,Actief,Netwerkevenement,https://www.voka.be/activiteiten/HRmidzomerbar,Offline
3,5F62AF96-0E2D-ED11-9DB1-000D3A211D18,22-OV-01-0163,2022-10-07 15:00:00,OV-Klantenarena,OV-Klantenarena,Nieuw,2022-10-07 10:30:00,Actief,Netwerkevenement,None,Offline
4,607FD2BC-CC2D-ED11-9DB1-000D3A211D18,22-OV-01-0164,2022-09-15 21:00:00,OV-Regioraad Vlaamse Ardennen - 15 september 2022,OV-Regioraad Vlaamse Ardennen - 15 september 2022,Nieuw,2022-09-15 18:00:00,Actief,Netwerkevenement,None,Offline
...,...,...,...,...,...,...,...,...,...,...,...
4096,AF74939A-4E02-E211-8D2C-984BE17C47FB,12-OV-03-0045,2012-12-18 00:00:00,OV-Levenscyclusanalyse: hoe toepassen in uw be...,OV-Levenscyclusanalyse: hoe toepassen in uw be...,Nieuw,2012-12-18 00:00:00,Actief,Projectgebonden,None,Offline
4097,497A08DE-3EDA-E111-8D88-984BE17C47FB,12-OV-01-0061,2012-09-26 00:00:00,OV-NW-L&L-Prof. Verhaeghe en Van Doorselaere -...,OV-NW-L&L-Prof. Verhaeghe en Van Doorselaere -...,Nieuw,2012-09-26 00:00:00,Actief,Netwerkevenement,None,Offline
4098,0BD4F6A1-7BEB-E111-8D88-984BE17C47FB,12-OV-03-0041,2012-09-18 00:00:00,OV-Seminarie Open Innovatie & IP,OV-Seminarie Open Innovatie & IP,Nieuw,2012-09-18 00:00:00,Actief,Projectgebonden,None,Offline
4099,D1EF1C5C-21ED-E111-8D88-984BE17C47FB,12-OV-01-0064,2012-09-18 00:00:00,OV-NW-Voka Visit Volvo Cars (Gent),OV-NW-Voka Visit Volvo Cars (Gent),Nieuw,2012-09-18 00:00:00,Actief,Netwerkevenement,None,Offline


In [11]:
visit_df = pd.read_sql("SELECT * FROM cdi_visit", conn)
visit_df

,crm_cdi_visit_adobe_reader,crm_cdi_visit_bounce,crm_cdi_visit_browser,crm_cdi_visit_campagne_code,crm_campagne_id,crm_cdi_visit_ip_stad,crm_cdi_visit_ip_company,crm_persoon_id,crm_cdi_visit_contact_naam,crm_cdi_visit_containssocialprofile,...,crm_cdi_visit_referring_host,crm_cdi_visit_score,crm_cdi_visit_referrer_type,crm_cdi_visit_started_on,crm_cdi_visit_ip_status,crm_cdi_visit_time,crm_cdi_visit_total_pages,crm_cdi_visit_id,crm_cdi_visit_aangemaakt_op,crm_cdi_visit_gewijzigd_op


In [12]:
web_content_df = pd.read_sql("SELECT * FROM cdi_web_content", conn)
web_content_df

,crm_cdi_webcontent_campaign,crm_cdi_webcontent_campaign_name,crm_cdi_webcontent_name,crm_cdi_webcontent_web_content,crm_cdi_webcontent_gemaakt_door_naam,crm_cdi_webcontent_created_on,crm_cdi_webcontent_gewijzigd_door_naam,crm_cdi_webcontent_modified_on,crm_cdi_webcontent_owner,crm_cdi_webcontent_owner_name,crm_cdi_webcontent_het_bezitten_van_business_unit
0,A63991CD-BE97-EC11-B400-002248800675,"VB - ""Lab Circulaire Economie"" start op 29.04....",Enquête - Sessie 1: De opportuniteiten van cir...,9C7A0A4A-E8C9-EC11-A7B5-000D3A20A90F,"Oinats, Silke",2022-05-02 09:20:06,"Oinats, Silke",2022-05-02 09:20:37,9F72095B-1B4C-E111-8DCE-18A905732A15,"Oinats, Silke",436CCFE0-5E72-E511-9AB0-005056B06EB4
1,A0F4B990-BD0F-EC11-8126-001DD8B72B62,LI-P-GROEP 103-PP.NextGen II-Plato 2021,LI - EO - Plato Nextgen II - Sessie 5 - 5/05/2022,0D8F5E7F-26CA-EC11-A7B5-000D3A20A90F,"Jonkmans, Lien",2022-05-02 16:45:28,"Jonkmans, Lien",2022-05-02 16:53:37,B1F841AD-38BA-EB11-8122-001DD8B72B62,"Jonkmans, Lien",CA3A0E7A-244B-E111-8DCE-18A905732A15
2,AB3A6E75-9A96-EA11-8111-001DD8B72B62,FBC: Wuco Raad van Advies,LI - EO - Evaluatie adviseurs Wuco,D09B1F21-AFCA-EC11-A7B5-000D3A20A90F,"Jonkmans, Lien",2022-05-03 09:03:29,"Jonkmans, Lien",2022-05-03 09:03:51,B1F841AD-38BA-EB11-8122-001DD8B72B62,"Jonkmans, Lien",CA3A0E7A-244B-E111-8DCE-18A905732A15
3,767D9750-5C2B-EC11-8124-001DD8B72B61,"VB - ""Beyond Borders Frankrijk"" start 19.04.20...",Enquête - Sessie 3: Juridische aspecten & fina...,AC20F97F-3ECC-EC11-A7B5-000D3A20A90F,"Oinats, Silke",2022-05-05 08:42:16,"Oinats, Silke",2022-05-05 08:42:53,9F72095B-1B4C-E111-8DCE-18A905732A15,"Oinats, Silke",436CCFE0-5E72-E511-9AB0-005056B06EB4
4,624E37F6-785E-EC11-8F8F-000D3A2BC8AD,"LI - Leer een douaneaangifte lezen, analyseren...","LI - IO - Leer een douaneaangifte lezen, anal...",E0CFB2E2-4ACC-EC11-A7B5-000D3A20A90F,"Vranken, Sanne",2022-05-05 10:11:03,"Vranken, Sanne",2022-05-05 10:11:43,1752AA3C-AABF-EB11-8120-001DD8B72B61,"Vranken, Sanne",CA3A0E7A-244B-E111-8DCE-18A905732A15
...,...,...,...,...,...,...,...,...,...,...,...
4068,8A674D06-237A-ED11-81AD-6045BD895D85,"VB - ""Cybersecurity Roadshow Green Energy Park...","Enquête - ""Cybersecurity Roadshow Green Energy...",CAC03A46-06E0-ED11-A7C6-6045BD974EB2,"Oinats, Silke",2023-04-21 07:35:17,"Oinats, Silke",2023-04-21 07:35:47,9F72095B-1B4C-E111-8DCE-18A905732A15,"Oinats, Silke",436CCFE0-5E72-E511-9AB0-005056B06EB4
4069,5C51AE39-DB17-ED11-B83E-000D3AAB2E73,LI - NW Jong Voka -Jongeren werkingsjaar 2022-...,LI - NW: Evaluatie Jong Voka Sport: padeltoern...,A21DE96A-EFE8-ED11-A7C6-6045BD974EB2,"Kuijpers, Laurens",2023-05-02 15:44:14,"Kuijpers, Laurens",2023-05-02 15:44:51,E98EF42B-DDB6-E611-80E4-001DD8B72B61,"Kuijpers, Laurens",CA3A0E7A-244B-E111-8DCE-18A905732A15
4070,C3ACE794-77AC-ED11-AAD0-6045BD895BFB,LI - Omzetbooster 2023,LI - EO - Enquête Omzetbooster 2023 - 26 en 27...,54275AFE-6EE2-ED11-A7C7-6045BD974EB2,"Put, Laura",2023-04-24 09:09:53,"Put, Laura",2023-04-24 09:41:47,47E5C3FA-419E-ED11-AAD1-6045BD8952CE,"Put, Laura",CA3A0E7A-244B-E111-8DCE-18A905732A15
4071,0DB7EFB2-497D-ED11-81AD-6045BD895D85,"VB - ""miK Duurzaamheid en verpakken"" op 25.04....","Enquête - ""miK Duurzaamheid en verpakken""",796D4200-02E4-ED11-A7C7-6045BD974EB2,"Oinats, Silke",2023-04-26 09:14:38,"Oinats, Silke",2023-04-26 09:18:41,9F72095B-1B4C-E111-8DCE-18A905732A15,"Oinats, Silke",436CCFE0-5E72-E511-9AB0-005056B06EB4


In [16]:
persoon_df = pd.read_sql("SELECT * FROM persoon", conn)
persoon_df

,crm_persoon_id,crm_persoon_persoonsnr,crm_persoon_reden_van_status,crm_persoon_web_login,crm_persoon_mail_regio_antwerpen_waasland,crm_persoon_mail_regio_brussel_hoofdstedelijk_gewest,crm_persoon_mail_regio_limburg,crm_persoon_mail_regio_mechelen_kempen,crm_persoon_mail_regio_oost_vlaanderen,crm_persoon_mail_regio_vlaams_brabant,...,crm_persoon_mail_thema_strategie_en_algemeen_management,crm_Persoon_Mail_thema_talent,crm_persoon_mail_thema_welzijn,crm_persoon_mail_type_bevraging,crm_persoon_mail_type_communities_en_projecten,crm_persoon_mail_type_netwerkevenementen,crm_persoon_mail_type_nieuwsbrieven,crm_persoon_mail_type_opleidingen,crm_persoon_mail_type_persberichten_belangrijke_meldingen,crm_persoon_marketingcommunicatie
0,4B834B8A-BF61-4939-8F6A-00000632F4B5,415561,Inactief,None,Nee,Nee,Nee,Nee,Nee,Nee,...,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Flexibel
1,581776D7-3F67-4789-98E2-0003862CBBCB,403884,Actief,None,Nee,Nee,Nee,Nee,Nee,Nee,...,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Flexibel
2,854EFBA1-B868-4E37-94D8-00042C492165,415749,Actief,None,Nee,Nee,Nee,Nee,Nee,Nee,...,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Flexibel
3,83D2028C-63C8-EC11-A7B5-000D3A20A90F,557484,Actief,eddy.patfoort@telenet.be,Nee,Nee,Nee,Nee,Nee,Nee,...,Ja,Nee,Nee,Nee,Nee,Ja,Ja,Nee,Nee,Strikt
4,EE4A8BC8-DCC9-EC11-A7B5-000D3A20A90F,557485,Actief,None,Nee,Nee,Nee,Nee,Nee,Nee,...,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Strikt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583609,98C462F5-B47E-4A6A-B92B-FF8584631F0E,395769,Actief,None,Nee,Nee,Nee,Nee,Nee,Nee,...,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Flexibel
583610,1A398F62-2046-4C8C-9F19-FF85D944CC22,380915,Actief,None,Nee,Nee,Nee,Nee,Nee,Nee,...,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Flexibel
583611,9D98E6D2-B52C-4520-920F-FFB721DCB004,398971,Actief,None,Nee,Nee,Nee,Nee,Nee,Nee,...,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Flexibel
583612,38BE0C92-8CA8-4F91-A90A-FFDADC2BDDDC,394580,Actief,None,Nee,Nee,Nee,Nee,Nee,Nee,...,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Nee,Uitgeschreven


In [13]:
table_info = get_db_info("cdi_visit", info)
visit_from_csv =  get_new_data("CDI visits.csv", table_info)
visit_from_csv

,crm_CDI_Visit_Adobe_Reader,crm_CDI_Visit_Bounce,crm_CDI_Visit_Browser,crm_CDI_Visit_Campagne_Code,crm_CDI_Visit_Campaign,crm_CDI_Visit_IP_Stad,crm_CDI_Visit_IP_Company,crm_CDI_Visit_Contact,crm_CDI_Visit_Contact_Naam_,crm_CDI_Visit_containssocialprofile,...,crm_CDI_Visit_Referring_Host,crm_CDI_Visit_Score,crm_CDI_Visit_Referrer_Type,crm_CDI_Visit_Started_On,crm_CDI_Visit_IP_Status,crm_CDI_Visit_Time,crm_CDI_Visit_Total_Pages,crm_CDI_Visit_Visit,crm_CDI_Visit_Aangemaakt_op,crm_CDI_Visit_Gewijzigd_op
0,Ja,Nee,SAFARI,NaN,2E3181C5-92E8-EB11-8121-001DD8B72B61,Antwerp,Telenet,5FC1E5CB-A86B-E111-B43A-00505680000A,"Poels, Dirk",Nee,...,NaN,5,E-mail,7-11-2021 06:16:41,VLG,11-07-2021 06:16:41 (UTC),2,92B18E10-6870-4046-A0DE-E3A09A85C049,7-11-2021 07:16:42,28-10-2023 17:21:35
1,Nee,Ja,NaN,NaN,NaN,Weerde,Telenet,10D29432-805E-EC11-8F8F-000D3A2BC671,"Petré, Ulrich",Nee,...,www.linkedin.com,4,E-mail,26-3-2022 06:50:50,VLG,03-26-2022 06:50:50 (UTC),1,4AD39F7F-A623-4730-A51C-6DEF3EFE9C06,26-3-2022 07:51:12,14-10-2023 16:39:55
2,Nee,Ja,NaN,NaN,D812EBE5-0768-EE11-9AE7-6045BD895D85,Jabbeke,Telenet,0E97FD13-51C1-E311-9A5C-005056B06EB4,"Van Wesemael, Jan",Nee,...,www.linkedin.com,4,E-mail,21-10-2023 11:11:20,VLG,10-21-2023 11:11:20 (UTC),1,E7C45C89-9E8E-4AF6-8229-1A2D71FA3CB1,24-10-2023 02:57:12,24-10-2023 02:57:14
3,Nee,Ja,ANDROID,NaN,NaN,NaN,NaN,086113C8-3D73-E111-B43A-00505680000A,"Tuybens, Dieter",Nee,...,NaN,4,E-mail,8-9-2023 07:50:48,NaN,09-08-2023 07:50:48 (UTC),1,808370D9-46E0-48BD-A074-DBA8123B99CC,8-9-2023 09:51:06,14-10-2023 16:47:36
4,Nee,Ja,ANDROID,NaN,NaN,NaN,NaN,502642E5-5CC9-E411-81DA-005056B06EB4,"Martens, Els",Nee,...,NaN,4,E-mail,15-9-2023 06:04:10,NaN,09-15-2023 06:04:10 (UTC),1,E765C779-0DA4-4C6D-A05E-F6BC77EFEA4B,15-9-2023 08:04:12,7-10-2023 16:33:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62260,Nee,Nee,SAFARI,NaN,FD0E0E2C-DF7A-ED11-81AD-6045BD895D85,Halle,Telenet,DA2B3EB4-A41B-E811-80F0-001DD8B72B61,"Verhoeven, Michele",Nee,...,NaN,14,E-mail,6-2-2023 13:06:30,VLG,02-06-2023 13:06:30 (UTC),11,2FA5FA7A-A2B9-4D68-88D0-88099A32C047,13-2-2023 02:45:07,21-10-2023 16:33:33
62261,Nee,Nee,SAFARI,NaN,FDDD8E61-4AA2-ED11-AAD1-6045BD895B5A,Wilsele,Belgacom SA,B9FF45B9-AB4C-E511-B697-005056B06EB4,"Cox, Pieter",Nee,...,www.voka.be,10,Onbekend,27-2-2023 10:33:42,VLG,02-27-2023 10:34:10 (UTC),7,C1DB5C01-8E84-4CAB-B4E8-98E57FC5790A,27-2-2023 17:05:26,7-10-2023 16:31:34
62262,Nee,Nee,SAFARI,NaN,FE2BCE78-CCC3-ED11-83FF-6045BD8950C6,Leuven,PRO,5A9B8938-FC4D-EE11-BE6E-6045BD895FE4,"Mihoc, Claudia",Nee,...,NaN,7,Direct verkeer,18-10-2023 14:18:50,VLG,10-18-2023 14:19:10 (UTC),2,41387218-1436-43B0-9A2D-019D4648363B,18-10-2023 17:49:07,18-10-2023 17:49:44
62263,Nee,Nee,SAFARI,NaN,FE3D13D2-7B1B-EC11-8123-001DD8B72B61,Vilvoorde,Telenet,D01F4444-8A51-EB11-811A-001DD8B72B62,"Daniëls, Petra",Nee,...,NaN,6,E-mail,1-11-2021 09:18:50,VLG,11-01-2021 09:18:50 (UTC),3,46952633-0996-4700-B400-CD28846015A3,1-11-2021 10:19:26,7-10-2023 17:25:55


In [14]:
# Merge visit and campaign
visit_df = pd.merge(
    visit_from_csv,
    campaign_df,
    how="inner",
    left_on="crm_CDI_Visit_Campaign",
    right_on="crm_campagne_id",
)
visit_df

,crm_CDI_Visit_Adobe_Reader,crm_CDI_Visit_Bounce,crm_CDI_Visit_Browser,crm_CDI_Visit_Campagne_Code,crm_CDI_Visit_Campaign,crm_CDI_Visit_IP_Stad,crm_CDI_Visit_IP_Company,crm_CDI_Visit_Contact,crm_CDI_Visit_Contact_Naam_,crm_CDI_Visit_containssocialprofile,...,crm_campagne_campagne_nr,crm_campagne_einddatum,crm_campagne_naam,crm_campagne_naam_in_email,crm_campagne_reden_van_status,crm_campagne_startdatum,crm_campagne_status,crm_campagne_type_campagne,crm_campagne_url_voka_be,crm_campagne_soort_campagne
0,Nee,Ja,ANDROID,NaN,071FBE05-F261-EE11-8DF0-6045BD974EB2,Brussels,Mobistar Enterprise Services,0D3DACB1-D44E-E911-80FC-001DD8B72B62,"Staessen, Joeri",Nee,...,23-OV-01-0167,2023-10-18 13:30:00,OV-V@V-AIO-Webinar Kinderopvang,"Webinar Kinderopvang, wat kan je als bedrijf z...",Voltooid,2023-10-18 12:00:00,Actief,Netwerkevenement,https://www.voka.be/activiteiten/webinar-kinde...,Online
1,Nee,Ja,CHROME,NaN,071FBE05-F261-EE11-8DF0-6045BD974EB2,Aalst,Telenet Operaties N.V.,731D4461-2C19-E711-80E4-001DD8B72B62,"Valckenier, Silke",Nee,...,23-OV-01-0167,2023-10-18 13:30:00,OV-V@V-AIO-Webinar Kinderopvang,"Webinar Kinderopvang, wat kan je als bedrijf z...",Voltooid,2023-10-18 12:00:00,Actief,Netwerkevenement,https://www.voka.be/activiteiten/webinar-kinde...,Online
2,Nee,Ja,CHROME,NaN,071FBE05-F261-EE11-8DF0-6045BD974EB2,Antwerp,Mobistar Enterprise Services,7F393295-EB37-E811-80F0-001DD8B72B61,"Van den Brande, Koen",Nee,...,23-OV-01-0167,2023-10-18 13:30:00,OV-V@V-AIO-Webinar Kinderopvang,"Webinar Kinderopvang, wat kan je als bedrijf z...",Voltooid,2023-10-18 12:00:00,Actief,Netwerkevenement,https://www.voka.be/activiteiten/webinar-kinde...,Online
3,Nee,Ja,CHROME,NaN,071FBE05-F261-EE11-8DF0-6045BD974EB2,Beerlegem,Telenet N.V.,7EB669AA-8229-E211-B2FD-005056B06EB4,"Verbeke, Isabelle",Nee,...,23-OV-01-0167,2023-10-18 13:30:00,OV-V@V-AIO-Webinar Kinderopvang,"Webinar Kinderopvang, wat kan je als bedrijf z...",Voltooid,2023-10-18 12:00:00,Actief,Netwerkevenement,https://www.voka.be/activiteiten/webinar-kinde...,Online
4,Nee,Ja,CHROME,NaN,071FBE05-F261-EE11-8DF0-6045BD974EB2,Boechout,Telenet,E68A075E-1452-EE11-BE6D-6045BD974EB2,"Hilkens, Christophe",Nee,...,23-OV-01-0167,2023-10-18 13:30:00,OV-V@V-AIO-Webinar Kinderopvang,"Webinar Kinderopvang, wat kan je als bedrijf z...",Voltooid,2023-10-18 12:00:00,Actief,Netwerkevenement,https://www.voka.be/activiteiten/webinar-kinde...,Online
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148,Nee,Nee,IPHONE,NaN,98B952D1-BEC4-EC11-A7B6-000D3A497E09,Meerbeke,Belgacom SA,B2DC2DF8-539C-EB11-811E-001DD8B72B62,"De Decker, Michael",Nee,...,22-OV-01-0097,2022-06-11 23:00:00,OV-NW-Rode Loper Selectie-House of Cohen-Vooru...,"House of Cohen, Songs of Leonard Cohen",Nieuw,2022-06-11 20:30:00,Actief,Netwerkevenement,https://www.voka.be/activiteiten/house-cohen-v...,Offline
4149,Nee,Nee,IPHONE,NaN,B20D19A9-F1CC-EB11-8120-001DD8B72B61,Saint-Gilles,Telenet Group BVBA/SPRL,FD063CC3-41F3-EA11-8115-001DD8B72B62,"Synhaeve, Inya",Nee,...,21-OV-02-0118,2021-12-13 12:00:00,OV-BTW Class: Btw-valkuilen bij internationale...,Btw-valkuilen bij internationale handel,Nieuw,2021-12-13 08:30:00,Actief,Opleiding,https://www.voka.be/activiteiten/btw-class-btw...,Online
4150,Nee,Nee,IPHONE,NaN,C6849948-6553-EC11-8C62-000D3ABFCF4A,Ghent,Telenet,2C4F2668-B21E-E311-8A47-005056B06EC4,"David, Etienne",Nee,...,22-OV-02-0006,2022-03-25 16:00:00,OV-Presentatietraining met Wim De Vilder,Presentatietraining met Wim De Vilder,Nieuw,2022-03-25 13:00:00,Actief,Opleiding,https://www.voka.be/activiteiten/presentatietr...,Offline
4151,Nee,Nee,SAFARI,NaN,7AF4B267-F115-EC11-8126-001DD8B72B62,Ingelmunster,Belgacom SA,8C545FEE-F20A-EC11-8123-001DD8B72B61,"Pynaert, Tom",Nee,...,22-OV-04-0008,2023-04-18 16:30:00,OV-P-Groep 324-PP.Explore-Plato-2022,Plato Explore,Nieuw,2022-05-19 13:00:00,Actief,Project,http://voka.ddev.site/activiteiten/plato-explore,Offline


In [15]:
visit_df = pd.merge(
    visit_from_csv,
    contact_df,
    how="inner",
    left_on="crm_CDI_Visit_Contact",
    right_on="crm_contact_id",
)
visit_df

,crm_CDI_Visit_Adobe_Reader,crm_CDI_Visit_Bounce,crm_CDI_Visit_Browser,crm_CDI_Visit_Campagne_Code,crm_CDI_Visit_Campaign,crm_CDI_Visit_IP_Stad,crm_CDI_Visit_IP_Company,crm_CDI_Visit_Contact,crm_CDI_Visit_Contact_Naam_,crm_CDI_Visit_containssocialprofile,...,crm_CDI_Visit_Total_Pages,crm_CDI_Visit_Visit,crm_CDI_Visit_Aangemaakt_op,crm_CDI_Visit_Gewijzigd_op,crm_contact_id,crm_contact_account,crm_contact_functietitel,crm_persoon_id,crm_contact_status,crm_contact_voka_medewerker
0,Ja,Nee,SAFARI,NaN,2E3181C5-92E8-EB11-8121-001DD8B72B61,Antwerp,Telenet,5FC1E5CB-A86B-E111-B43A-00505680000A,"Poels, Dirk",Nee,...,2,92B18E10-6870-4046-A0DE-E3A09A85C049,7-11-2021 07:16:42,28-10-2023 17:21:35,5FC1E5CB-A86B-E111-B43A-00505680000A,894F9148-B9B2-E411-9B05-005056B06EB4,Medewerker,33A111AF-2268-E111-A00F-00505680000A,Actief,0
1,Nee,Ja,IPHONE,NaN,2E3181C5-92E8-EB11-8121-001DD8B72B61,Hasselt,Destiny N.V / S.A,5FC1E5CB-A86B-E111-B43A-00505680000A,"Poels, Dirk",Nee,...,1,B0440A2E-CB14-47B2-BEA3-C3C05BD29322,7-11-2021 08:07:09,28-10-2023 17:21:33,5FC1E5CB-A86B-E111-B43A-00505680000A,894F9148-B9B2-E411-9B05-005056B06EB4,Medewerker,33A111AF-2268-E111-A00F-00505680000A,Actief,0
2,Nee,Ja,SAFARI,NaN,NaN,Waarschoot,Coditel Brabant SPRL/BVBA,5FC1E5CB-A86B-E111-B43A-00505680000A,"Poels, Dirk",Nee,...,1,765E1584-2F9A-4286-9ED0-C4A12BD13F04,6-10-2023 12:16:01,6-10-2023 12:16:03,5FC1E5CB-A86B-E111-B43A-00505680000A,894F9148-B9B2-E411-9B05-005056B06EB4,Medewerker,33A111AF-2268-E111-A00F-00505680000A,Actief,0
3,Nee,Ja,SAFARI,NaN,36B0D3F8-63EA-ED11-8849-6045BD895D78,Waarschoot,Coditel Brabant SPRL/BVBA,5FC1E5CB-A86B-E111-B43A-00505680000A,"Poels, Dirk",Nee,...,1,8ACDB099-2414-45B0-96F4-EB3C0A726058,23-10-2023 02:00:19,23-10-2023 02:00:23,5FC1E5CB-A86B-E111-B43A-00505680000A,894F9148-B9B2-E411-9B05-005056B06EB4,Medewerker,33A111AF-2268-E111-A00F-00505680000A,Actief,0
4,Nee,Ja,SAFARI,NaN,D43CF03F-4652-EE11-BE6D-6045BD895BFB,Waarschoot,Coditel Brabant SPRL/BVBA,5FC1E5CB-A86B-E111-B43A-00505680000A,"Poels, Dirk",Nee,...,1,37ABE058-F2C8-431A-82EC-1250E0368AFA,2-10-2023 08:08:27,2-10-2023 08:08:28,5FC1E5CB-A86B-E111-B43A-00505680000A,894F9148-B9B2-E411-9B05-005056B06EB4,Medewerker,33A111AF-2268-E111-A00F-00505680000A,Actief,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58792,Nee,Nee,SAFARI,NaN,F06DB77D-9C2F-EE11-BDF4-6045BD895273,Hulste,Coditel Brabant SPRL/BVBA,241D5522-F29C-EB11-811E-001DD8B72B62,"Matthys, Gino",Nee,...,5,D7EA55BF-8F76-4074-A356-2ED2880BC394,4-10-2023 10:40:32,4-10-2023 10:43:34,241D5522-F29C-EB11-811E-001DD8B72B62,09BF0814-AA68-E111-B43A-00505680000A,Operations manager,261D5522-F29C-EB11-811E-001DD8B72B62,Actief,0
58793,Nee,Nee,SAFARI,NaN,F312E4E3-7927-EC11-8127-001DD8B72B62,Denderleeuw,Belgacom SA,98C2FDA1-0ECD-E611-80E5-001DD8B72B61,"Vanhove, Tom",Nee,...,6,3BADCCCE-F5BF-4F40-BEE2-AF632CE949E8,10-11-2021 09:04:43,28-10-2023 17:20:28,98C2FDA1-0ECD-E611-80E5-001DD8B72B61,117012CE-9A99-E511-A092-005056B06EB4,Sales manager,33C0847D-0ECD-E611-80E5-001DD8B72B61,Actief,0
58794,Nee,Nee,SAFARI,NaN,F72CA043-E9E8-ED11-A7C6-6045BD895FE4,Ghent,Telenet N.V.,B7C123AC-6672-E111-B43A-00505680000A,"Van Vlaenderen, Dirk",Nee,...,2,2D81B565-3D9D-40C9-BFBF-A9ED64EBCD2E,12-10-2023 09:47:03,12-10-2023 09:47:07,B7C123AC-6672-E111-B43A-00505680000A,F988DCBA-C768-E111-B43A-00505680000A,Algemeen directeur,BB6A0310-1468-E111-A00F-00505680000A,Actief,0
58795,Nee,Nee,SAFARI,NaN,FCB85C27-462C-EC11-8127-001DD8B72B62,Zingem,PROXIMUS,21464A07-006C-E111-B43A-00505680000A,"Van Oorlé, Joakim",Nee,...,3,4316227F-DBC3-4D03-BD18-A56C393C0759,30-10-2021 10:09:14,28-10-2023 17:29:44,21464A07-006C-E111-B43A-00505680000A,9AE855E4-C068-E111-B43A-00505680000A,Zaakvoerder,EA63A4FF-4E69-E111-B43A-00505680000A,Actief,0


In [21]:
# Merge visit and persoon
visit_df = pd.merge(
    visit_from_csv,
    persoon_df,
    how="inner",
    left_on="crm_CDI_Visit_Contact",
    right_on="crm_persoon_id",
)
visit_df

,crm_CDI_Visit_Adobe_Reader,crm_CDI_Visit_Bounce,crm_CDI_Visit_Browser,crm_CDI_Visit_Campagne_Code,crm_CDI_Visit_Campaign,crm_CDI_Visit_IP_Stad,crm_CDI_Visit_IP_Company,crm_CDI_Visit_Contact,crm_CDI_Visit_Contact_Naam_,crm_CDI_Visit_containssocialprofile,...,crm_persoon_mail_thema_strategie_en_algemeen_management,crm_Persoon_Mail_thema_talent,crm_persoon_mail_thema_welzijn,crm_persoon_mail_type_bevraging,crm_persoon_mail_type_communities_en_projecten,crm_persoon_mail_type_netwerkevenementen,crm_persoon_mail_type_nieuwsbrieven,crm_persoon_mail_type_opleidingen,crm_persoon_mail_type_persberichten_belangrijke_meldingen,crm_persoon_marketingcommunicatie
